In [1]:
import pandas as pd
import os

In [2]:
def get_ICS_severity(severity_table, hazards):
    severities = {hazard:[] for hazard in hazards}
    #severity table -> index
    for hazard in hazards:
        #temp_df = severity_table.loc[severity_table["Hazard"]==hazard.replace("_", " ")].reset_index(drop=True)
        severities[hazard].append(round(severity_table.at[hazard,"Average Injuries"]) - severity_table.at[hazard, "Total Avg Injuries"])
        severities[hazard].append(severity_table.at[hazard,"Average Structures Damaged"] - severity_table.at[hazard,"Total Avg Str Dam"])
        severities[hazard].append(severity_table.at[hazard,"Average Structures Destroyed"] - severity_table.at[hazard,"Total Avg Str Des"])
        severities[hazard].append(round(severity_table.at[hazard,"Average Fatalities"]) - severity_table.at[hazard, "Total Avg Fatalities"])
    severity_df = pd.DataFrame(severities, 
                            index=['Diff_Injuries', 'Diff_Structures_Damages', 
                                     'Diff_Structures_Destroyed','Diff_Fatalities'])
    total_severities = []
    curr_severities = {}
    for hazard in hazards: 
        injuries = severity_df.at['Diff_Injuries',hazard]
        str_dam = severity_df.at['Diff_Structures_Damages', hazard]
        str_des = severity_df.at['Diff_Structures_Destroyed', hazard]
        fatalities = severity_df.at['Diff_Fatalities', hazard]
        if injuries == 0 and fatalities == 0 and str_des == 0 and str_dam ==0:
             impact = "Minimal Impact"
        elif injuries <= 2 and fatalities == 0 and str_des <= 10 and str_dam <= 10:
             impact = "Minor Impact"
        elif injuries <= 2 and fatalities == 0 and (str_des > 10 or str_dam > 10):
             impact = "Major Impact"
        else:
            if fatalities<2:
                impact = 'Critical Impact'
            else: #fatalities>2
                impact = 'Catastrophic Impact'
        curr_severities[hazard] = impact
    return curr_severities

In [3]:
def get_SAFECOM_severity(severities):
    curr_severities = {hazard:0 for hazard in severities}
    for hazard in severities:
        s = severities[hazard]
        if s<=0.1: #negligible impact
            severity = 'Minimal Impact'
        elif s>0.1 and s <= 0.5:
            severity = 'Minor Impact'
        elif s>0.5 and s<=1:
            severity = 'Major Impact'
        elif s>1 and s<=2:
            severity = 'Critical Impact'
        elif s>2:
            severity = 'Catastrophic Impact'
        curr_severities[hazard] = severity
    return curr_severities

In [4]:
def get_likelihood(rates):
    curr_likelihoods = {hazard:0 for hazard in rates}
    for hazard in rates:
        r = rates[hazard]
        if r>=100:
            likelihood = 'Frequent'
        elif r>=10 and r<100:
            likelihood = 'Probable'
        elif r>=1 and r<10:
            likelihood = 'Remote'
        elif r>=0.1 and r<1:
            likelihood = 'Extremely Remote'
        elif r<0.1:
            likelihood = 'Extremely Improbable'
        curr_likelihoods[hazard] = likelihood
    return curr_likelihoods

In [5]:
def get_UAS_likelihood(frequency):
    curr_likelihoods = {hazard:0 for hazard in frequency}
    for hazard in frequency:
        r = frequency[hazard]
        if r==5:
            likelihood = 'Frequent'
        elif r==4:
            likelihood = 'Probable'
        elif r==3:
            likelihood = 'Remote'
        elif r==2:
            likelihood = 'Extremely Remote'
        elif r==1:
            likelihood = 'Extremely Improbable'
        curr_likelihoods[hazard] = likelihood
    return curr_likelihoods

In [6]:
def get_risk_level(curr_severities, curr_likelihoods):
    curr_risks ={hazard:'' for hazard in curr_severities}
    risk_dict = {0:"Low", 5: "Medium", 10: "High"}
    df = pd.DataFrame([[0, 5, 10, 10, 10], [0, 5, 5, 10, 10], [0, 0, 5, 5, 10],
            [0, 0, 0, 5, 5], [0, 0, 0, 0, 5]],
          columns=['Minimal Impact', 'Minor Impact', 'Major Impact', 'Critical Impact', 'Catastrophic Impact'],
           index=['Frequent', 'Probable', 'Remote','Extremely Remote', 'Extremely Improbable'])
    for hazard in curr_risks:
        curr_risks[hazard] = risk_dict[df.at[curr_likelihoods[hazard], curr_severities[hazard]]]
    return curr_risks

In [7]:
def calc_severity_likelihood_risk(df, hazards, sev_col, rate_col, data_type):
    if data_type == "ICS":
        rates = {hazard: df.at[hazard, rate_col] for hazard in hazards}
        curr_severities = get_ICS_severity(df, hazards)
        curr_likelihoods = get_likelihood(rates)
    elif data_type == "SAFECOM":
        severities = {hazard: df.at[hazard, sev_col] for hazard in hazards}
        rates = {hazard: df.at[hazard, rate_col] for hazard in hazards}
        curr_severities = get_SAFECOM_severity(severities)
        curr_likelihoods = get_likelihood(rates)
    elif data_type =="SAFECOM_UAS":
        frequency = {hazard: df.at[hazard, rate_col] for hazard in hazards}
        severities = {hazard: df.at[hazard, sev_col] for hazard in hazards}
        curr_likelihoods = get_UAS_likelihood(frequency)
        curr_severities = get_SAFECOM_severity(severities)
    curr_risks = get_risk_level(curr_severities, curr_likelihoods)
    results = pd.DataFrame({"Hazard": hazards,
                           "Likelihood": [curr_likelihoods[h] for h in hazards],
                            "Severity": [curr_severities[h] for h in hazards],
                            "Risk": [curr_risks[h] for h in hazards]
                           })
    return results

In [8]:
#read in ICS file with hazards, rates, and severities
ICS = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()),'results','ICS_hazards.csv'), index_col='Hazard Name')
ICS_sev= None
ICS_rate = 'Average Occurrences per year'
ICS_hazards = list(ICS.index)

In [9]:
calc_severity_likelihood_risk(ICS, ICS_hazards, ICS_sev, ICS_rate, 'ICS').sort_values(by='Hazard').reset_index(drop=True)#.to_csv("ICS_hazards.csv")

,Hazard,Likelihood,Severity,Risk
0,Aerial Grounding,Probable,Critical Impact,High
1,Command Transitions,Frequent,Minor Impact,Medium
2,Cultural Resources,Probable,Minor Impact,Medium
3,Dry Weather,Probable,Minor Impact,Medium
4,Ecological,Probable,Critical Impact,High
5,Evacuations,Frequent,Major Impact,High
6,Extreme Weather,Frequent,Minor Impact,Medium
7,Floods,Remote,Critical Impact,Medium
8,Hazardous Terrain,Frequent,Minor Impact,Medium
9,Inaccurate Mapping,Probable,Minor Impact,Medium


In [10]:
#read in SAFECOM manned data with hazards, rates, and severities
SAFECOM = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), 'results', 'SAFECOM_hazards.csv'), index_col='Hazards')
SAFECOM_sev = 'Severity'
SAFECOM_rate = 'Rate'
SAFECOM_hazards = list(SAFECOM.index)

In [11]:
calc_severity_likelihood_risk(SAFECOM, SAFECOM_hazards, SAFECOM_sev, SAFECOM_rate, "SAFECOM").sort_values(by='Hazard').reset_index(drop=True)#.to_csv("SAFECOM_hazards.csv")

,Hazard,Likelihood,Severity,Risk
0,Airspace Control Hazards,Remote,Minimal Impact,Low
1,Avionics Malfunction,Probable,Minor Impact,Medium
2,Bucket Drop Failure,Probable,Minor Impact,Medium
3,Cargo Letdown Failure,Probable,Minor Impact,Medium
4,Control Surface Damage,Probable,Major Impact,Medium
5,Door Failure,Probable,Major Impact,Medium
6,Engine Malfunction,Probable,Minimal Impact,Low
7,Fuel System Malfunction,Remote,Minimal Impact,Low
8,Helitorch Operations Failure,Remote,Minor Impact,Low
9,Hydraulic Fluid Leak,Remote,Minor Impact,Low


In [12]:
#read in SAFECOM unmanned data with hazards, rates, severities
UAS_SAFECOM = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), 'results', 'safecom_fmea.csv'), index_col=0)
UAS_SAFECOM_sev = 'Severity'
UAS_SAFECOM_rate = 'Frequency'
UAS_SAFECOM_hazards = list(UAS_SAFECOM.index)

In [13]:
calc_severity_likelihood_risk(UAS_SAFECOM, UAS_SAFECOM_hazards, UAS_SAFECOM_sev, UAS_SAFECOM_rate, "SAFECOM_UAS")

,Hazard,Likelihood,Severity,Risk
0,Battery,Extremely Remote,Minor Impact,Low
1,Casing dislodged,Extremely Remote,Minimal Impact,Low
2,Communications/flight navigation with UAS failed,Extremely Remote,Minimal Impact,Low
3,Employee personal drone usage without certific...,Extremely Remote,Minimal Impact,Low
4,Engine Failure,Remote,Minor Impact,Low
5,Fight plan error,Remote,Minor Impact,Low
6,Hang Fire,Extremely Improbable,Minimal Impact,Low
7,High wind pushes UAS off course,Extremely Remote,Minor Impact,Low
8,Hobbyist offers services,Extremely Improbable,Minimal Impact,Low
9,Inappropriate Landing Site,Extremely Improbable,Major Impact,Low
